In [3]:
import numpy as np
import pandas as pd

In [22]:
data1 = pd.read_csv("../data/AAPL_05222012_0930_1300_LOB_2.csv")

In [23]:
# column name
col1 = list(data1.columns.values)[2:62]
len(col1)
col2 = [i[16:] for i in col1]
new_names = list(data1.columns.values)[0:2]+ [i[:-19] for i in col2]
data1.columns = new_names

# delete times
data1 = data1.head(203350)

In [75]:
# feature 2
level = np.arange(10)+1
bid_P = "BID_PRICE"
ask_P = "ASK_PRICE"
ask_V = "BID_SIZE"
bid_V = "ASK_SIZE"

def feature2_1(dataset):
    ret = pd.DataFrame()
    for i in level:
        ret["diff_ab" + str(i)] = data1[bid_P + str(i)] - data1[ask_P + str(i)]
    return ret

def feature2_2(dataset):
    ret = pd.DataFrame()
    for i in level:
        ret["mid_ab" + str(i)] = (data1[bid_P + str(i)]+data1[ask_P + str(i)])/2
    return ret

temp = feature2_2(data1)
#type(feature2_1(data1)[1][1])

In [101]:
# labels
# Mid-price
def label_midprice(dataset,step):
    tmp =  dataset["mid_ab1"].diff(periods = step)
    Y_midprice = pd.Series("stationary", index= range(dataset.shape[0]))
    Y_midprice[tmp>0] = "upward"
    Y_midprice[tmp<0] = "downward"
    return Y_midprice
    
#label_midprice(temp,step)

#Bid-ask spread crossing
def label_spread(dataset, step):
    bid_t2 = pd.Series(0, index= range(dataset.shape[0]))
    bid_t2[0:-step] = dataset["BID_PRICE1"][step:]
    indc1 = bid_t2 - dataset["ASK_PRICE1"]
    
    ask_t2 = pd.Series(0, index= range(dataset.shape[0]))
    ask_t2[0:-step] = dataset["ASK_PRICE1"][step:]
    indc2 = ask_t2 - dataset["BID_PRICE1"]
    Y_spread = pd.Series("stationary", index= range(dataset.shape[0]))
    Y_spread[indc1>0] = "upward"
    Y_spread[indc2<0] = "downward"
    Y_spread[-step:] = "NA"
    return sum(Y_spread=="downward"),sum(Y_spread=="upward")

#label_spread(data1,100 ) #1s
# question here

(13499, 13093)